# உருவாக்கும் நெட்வொர்க்குகள்

மீண்டும் மீண்டும் செயல்படும் நரம்பியல் நெட்வொர்க்குகள் (RNNs) மற்றும் அவற்றின் கதவுகள் கொண்ட செல்கள், உதாரணமாக நீண்ட மற்றும் குறுகிய கால நினைவுச் செல்கள் (LSTMs) மற்றும் கதவுகள் கொண்ட மீண்டும் செயல்படும் அலகுகள் (GRUs), மொழி மாதிரியாக்கத்திற்கான ஒரு முறை வழங்கின. அதாவது, அவை வார்த்தைகளின் வரிசையை கற்றுக்கொண்டு, வரிசையில் அடுத்த வார்த்தையை கணிக்க உதவுகின்றன. இதனால், RNNகளை **உருவாக்கும் பணிகளுக்கு** பயன்படுத்த முடிகிறது, உதாரணமாக சாதாரண உரை உருவாக்கல், இயந்திர மொழிபெயர்ப்பு, மற்றும் படங்களுக்கு விளக்கம் எழுதுதல்.

முந்தைய அலகில் RNN கட்டமைப்பைப் பற்றி நாம் விவாதித்தபோது, ஒவ்வொரு RNN அலகும் அடுத்த மறைமாநிலத்தை வெளியீடாக உருவாக்கியது. ஆனால், ஒவ்வொரு மீண்டும் செயல்படும் அலகுக்கும் மற்றொரு வெளியீட்டை சேர்க்க முடியும், இது ஒரு **வரிசையை** (மூல வரிசைக்கு சமமான நீளத்தில்) வெளியிட அனுமதிக்கும். மேலும், ஒவ்வொரு படியிலும் உள்ளீட்டை ஏற்காத RNN அலகுகளை பயன்படுத்த முடியும்; அவை ஒரு ஆரம்ப நிலை வெக்டரை மட்டும் எடுத்து, பின்னர் வெளியீடுகளின் வரிசையை உருவாக்கும்.

இந்த நோட்புக்கில், உரை உருவாக்க உதவும் எளிய உருவாக்கும் மாதிரிகளை நாங்கள் கவனிக்கப் போகிறோம். எளிமைக்காக, **எழுத்து மட்டத்திலான நெட்வொர்க்கை** உருவாக்குவோம், இது எழுத்து ஒன்றின் பின்னர் மற்றொன்றை உருவாக்கும். பயிற்சியின் போது, சில உரை தொகுப்பை எடுத்து, அதை எழுத்து வரிசைகளாக பிரிக்க வேண்டும்.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset,test_dataset,classes,vocab = load_dataset()

Loading dataset...
Building vocab...


## எழுத்து சொற்களஞ்சியத்தை உருவாக்குதல்

எழுத்து மட்டத்திலான உருவாக்க வலைப்பின்னலை உருவாக்க, உரையை சொற்களாக değil, தனிப்பட்ட எழுத்துகளாகப் பிரிக்க வேண்டும். இதை ஒரு வேறு டோக்கனைசரை வரையறுத்து செய்யலாம்:


In [2]:
def char_tokenizer(words):
    return list(words) #[word for word in words]

counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(char_tokenizer(line))
vocab = torchtext.vocab.vocab(counter)

vocab_size = len(vocab)
print(f"Vocabulary size = {vocab_size}")
print(f"Encoding of 'a' is {vocab.get_stoi()['a']}")
print(f"Character with code 13 is {vocab.get_itos()[13]}")

Vocabulary size = 82
Encoding of 'a' is 1
Character with code 13 is c


நாம் எவ்வாறு எங்கள் தரவுத்தொகுப்பிலிருந்து உரையை குறியாக்கம் செய்ய முடியும் என்பதற்கான உதாரணத்தை பார்ப்போம்:


In [3]:
def enc(x):
    return torch.LongTensor(encode(x,voc=vocab,tokenizer=char_tokenizer))

enc(train_dataset[0][1])

tensor([ 0,  1,  2,  2,  3,  4,  5,  6,  3,  7,  8,  1,  9, 10,  3, 11,  2,  1,
        12,  3,  7,  1, 13, 14,  3, 15, 16,  5, 17,  3,  5, 18,  8,  3,  7,  2,
         1, 13, 14,  3, 19, 20,  8, 21,  5,  8,  9, 10, 22,  3, 20,  8, 21,  5,
         8,  9, 10,  3, 23,  3,  4, 18, 17,  9,  5, 23, 10,  8,  2,  2,  8,  9,
        10, 24,  3,  0,  1,  2,  2,  3,  4,  5,  9,  8,  8,  5, 25, 10,  3, 26,
        12, 27, 16, 26,  2, 27, 16, 28, 29, 30,  1, 16, 26,  3, 17, 31,  3, 21,
         2,  5,  9,  1, 23, 13, 32, 16, 27, 13, 10, 24,  3,  1,  9,  8,  3, 10,
         8,  8, 27, 16, 28,  3, 28,  9,  8,  8, 16,  3,  1, 28,  1, 27, 16,  6])

## ஜெனரேட்டிவ் RNN பயிற்சி

RNN-ஐ உரை உருவாக்க பயிற்சி செய்வது இதுவே. ஒவ்வொரு படியிலும், `nchars` நீளமான எழுத்துக்களின் வரிசையை எடுத்து, ஒவ்வொரு உள்ளீட்டு எழுத்துக்கான அடுத்த வெளியீட்டு எழுத்தை உருவாக்க வலியுறுத்துவோம்:

![RNN மூலம் 'HELLO' என்ற வார்த்தையை உருவாக்கும் உதாரணத்தை காட்டும் படம்.](../../../../../translated_images/ta/rnn-generate.56c54afb52f9781d.webp)

உண்மையான சூழ்நிலைக்கு ஏற்ப, *end-of-sequence* `<eos>` போன்ற சில சிறப்பு எழுத்துக்களை சேர்க்க விரும்பலாம். எங்கள் நிலைமையில், முடிவில்லாத உரை உருவாக்கத்திற்காக நெட்வொர்க்கை பயிற்சி செய்ய விரும்புகிறோம், எனவே ஒவ்வொரு வரிசையின் அளவையும் `nchars` டோக்கன்களாக நிர்ணயிக்கிறோம். இதனால், ஒவ்வொரு பயிற்சி எடுத்துக்காட்டும் `nchars` உள்ளீடுகள் மற்றும் `nchars` வெளியீடுகளை (உள்ளீட்டு வரிசை ஒரு சின்னத்தை இடதுபுறம் நகர்த்தியது) கொண்டிருக்கும். மினிபேட்ச் பல இத்தகைய வரிசைகளைக் கொண்டிருக்கும்.

மினிபேட்ச்களை உருவாக்கும் முறை, `l` நீளமுள்ள ஒவ்வொரு செய்தி உரையையும் எடுத்து, அதிலிருந்து அனைத்து சாத்தியமான உள்ளீடு-வெளியீடு இணைப்புகளை உருவாக்குவது (அங்கு `l-nchars` இத்தகைய இணைப்புகள் இருக்கும்). அவை ஒரு மினிபேட்சை உருவாக்கும், மேலும் மினிபேட்ச்களின் அளவு ஒவ்வொரு பயிற்சி படியிலும் மாறுபடும்.


In [4]:
nchars = 100

def get_batch(s,nchars=nchars):
    ins = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    outs = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    for i in range(len(s)-nchars):
        ins[i] = enc(s[i:i+nchars])
        outs[i] = enc(s[i+1:i+nchars+1])
    return ins,outs

get_batch(train_dataset[0][1])

(tensor([[ 0,  1,  2,  ..., 28, 29, 30],
         [ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         ...,
         [20,  8, 21,  ...,  1, 28,  1],
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16]]),
 tensor([[ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         [ 2,  3,  4,  ...,  1, 16, 26],
         ...,
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16],
         [ 5,  8,  9,  ..., 27, 16,  6]]))

இப்போது ஜெனரேட்டர் நெட்வொர்க்கை வரையறுப்போம். இது முந்தைய யூனிட்டில் நாம் விவாதித்த எந்த ரிகரண்ட் செல் (எளிமையானது, LSTM அல்லது GRU) அடிப்படையாகக் கொண்டிருக்கலாம். எங்கள் உதாரணத்தில், நாம் LSTM ஐப் பயன்படுத்துவோம்.

நெட்வொர்க் எழுத்துக்களை உள்ளீடாக எடுப்பதால், மற்றும் சொற்கள்தொகை அளவு மிகவும் சிறியதாக இருப்பதால், எம்பெடிங் லேயர் தேவையில்லை, ஒன்-ஹாட்-என்கோடட் உள்ளீடு நேரடியாக LSTM செல்லுக்கு செல்லலாம். ஆனால், எழுத்து எண்களை உள்ளீடாக அனுப்புவதால், அவற்றை LSTM-க்கு அனுப்புவதற்கு முன் ஒன்-ஹாட்-என்கோடு செய்ய வேண்டும். இது `forward` பாஸ் போது `one_hot` செயல்பாட்டை அழைப்பதன் மூலம் செய்யப்படுகிறது. அவுட்புட் என்கோடர் என்பது ஒரு லினியர் லேயராக இருக்கும், இது ஹிடன் ஸ்டேட்டை ஒன்-ஹாட்-என்கோடட் அவுட்புட்டாக மாற்றும்.


In [5]:
class LSTMGenerator(torch.nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.rnn = torch.nn.LSTM(vocab_size,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, s=None):
        x = torch.nn.functional.one_hot(x,vocab_size).to(torch.float32)
        x,s = self.rnn(x,s)
        return self.fc(x),s

பயிற்சியின் போது, உருவாக்கப்பட்ட உரையை மாதிரியாக எடுக்க விரும்புகிறோம். இதைச் செய்ய, ஆரம்பத்தில் உள்ள string `start`-இல் இருந்து, `size` நீளத்துடன் ஒரு output string-ஐ உருவாக்கும் `generate` செயல்பாட்டை வரையறுக்கிறோம்.

இது செயல்படுவது பின்வருமாறு: முதலில், முழு `start` string-ஐ network-இல் கடத்தி, output state `s` மற்றும் அடுத்த கணிக்கப்பட்ட எழுத்து `out`-ஐ எடுப்போம். `out` ஒரு one-hot encoded வடிவில் இருக்கும், எனவே `argmax`-ஐ எடுத்து, அகராதியில் உள்ள எழுத்தின் குறியீடு `nc`-ஐ பெறுகிறோம். பின்னர், `itos`-ஐ பயன்படுத்தி உண்மையான எழுத்தை கண்டறிந்து, அதை `chars` எனப்படும் எழுத்துக்களின் பட்டியலில் சேர்க்கிறோம். இந்த ஒரு எழுத்தை உருவாக்கும் செயல்முறை `size` முறை மீண்டும் மீண்டும் செய்யப்படுகிறது, தேவையான எழுத்துக்களின் எண்ணிக்கையை உருவாக்க.


In [8]:
def generate(net,size=100,start='today '):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            nc = torch.argmax(out[0][-1])
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)

இப்போது பயிற்சியை தொடங்கலாம்! பயிற்சி சுழற்சி முந்தைய உதாரணங்களில் உள்ளதைப் போலவே இருக்கும், ஆனால் துல்லியத்தை அச்சிடுவதற்கு பதிலாக, 1000 எபோக்களுக்குப் பிறகு உருவாக்கப்பட்ட மாதிரித் உரையை அச்சிடுகிறோம்.

நாங்கள் இழப்பை கணக்கிடும் முறைக்கு சிறப்பு கவனம் செலுத்த வேண்டும். ஒரே-ஹாட்-என்கோடட் வெளியீடு `out` மற்றும் எதிர்பார்க்கப்படும் உரை `text_out` (எழுத்து குறியீடுகளின் பட்டியல்) கொடுக்கப்பட்டு இழப்பை கணக்கிட வேண்டும். அதிர்ஷ்டவசமாக, `cross_entropy` செயல்பாடு முதல் ஆஜராக சீரற்ற நெட்வொர்க் வெளியீட்டை எதிர்பார்க்கிறது, மற்றும் இரண்டாவது ஆஜராக வகுப்பு எண்ணை எதிர்பார்க்கிறது, இது நமக்கு இருப்பது தான். இது மினிபேட்ச் அளவின் மீது தானியங்கி சராசரி கணக்கீடையும் செய்கிறது.

மேலும், `samples_to_train` மாதிரிகளால் பயிற்சியை வரையறுக்கிறோம், அதிக நேரம் காத்திருக்க வேண்டாம் என்பதற்காக. நீண்ட பயிற்சியை முயற்சிக்கவும், பல எபோக்களுக்குப் பயிற்சி செய்யவும் உங்களை ஊக்குவிக்கிறோம் (அந்த சந்தர்ப்பத்தில், இந்தக் குறியீட்டின் சுற்றிலும் மற்றொரு சுழற்சியை உருவாக்க வேண்டும்).


In [9]:
net = LSTMGenerator(vocab_size,64).to(device)

samples_to_train = 10000
optimizer = torch.optim.Adam(net.parameters(),0.01)
loss_fn = torch.nn.CrossEntropyLoss()
net.train()
for i,x in enumerate(train_dataset):
    # x[0] is class label, x[1] is text
    if len(x[1])-nchars<10:
        continue
    samples_to_train-=1
    if not samples_to_train: break
    text_in, text_out = get_batch(x[1])
    optimizer.zero_grad()
    out,s = net(text_in)
    loss = torch.nn.functional.cross_entropy(out.view(-1,vocab_size),text_out.flatten()) #cross_entropy(out,labels)
    loss.backward()
    optimizer.step()
    if i%1000==0:
        print(f"Current loss = {loss.item()}")
        print(generate(net))

Current loss = 4.398899078369141
today sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr s
Current loss = 2.161320447921753
today and to the tor to to the tor to to the tor to to the tor to to the tor to to the tor to to the tor t
Current loss = 1.6722588539123535
today and the court to the could to the could to the could to the could to the could to the could to the c
Current loss = 2.423795223236084
today and a second to the conternation of the conternation of the conternation of the conternation of the 
Current loss = 1.702607274055481
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.692358136177063
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.9722288846969604
today and the control the control the control the control the control the control the control the control 
Current loss = 1.8

இந்த எடுத்துக்காட்டு ஏற்கனவே மிகவும் நல்ல உரையை உருவாக்குகிறது, ஆனால் இதை பல வழிகளில் மேலும் மேம்படுத்தலாம்:
* **சிறந்த மினிபேட்ச் உருவாக்கம்**. பயிற்சிக்கான தரவுகளை தயாரிக்கும் விதம் ஒவ்வொரு மாதிரியிலிருந்தும் ஒரு மினிபேட்ச் உருவாக்குவது ஆகும். இது சிறந்தது அல்ல, ஏனெனில் மினிபேட்ச்கள் அனைத்தும் வெவ்வேறு அளவுகளில் இருக்கும், மேலும் சிலவற்றை உருவாக்க முடியாது, ஏனெனில் உரை `nchars`-க்கு குறைவாக இருக்கும். மேலும், சிறிய மினிபேட்ச்கள் GPU-வை போதுமான அளவில் சுமக்க முடியாது. அனைத்து மாதிரிகளிலிருந்தும் ஒரு பெரிய உரைத் துண்டை எடுத்து, அனைத்து உள்ளீடு-வெளியீட்டு ஜோடிகளையும் உருவாக்கி, அவற்றை கலக்கி, சம அளவுடைய மினிபேட்ச்களை உருவாக்குவது புத்திசாலித்தனமாக இருக்கும்.
* **பல அடுக்கு LSTM**. 2 அல்லது 3 அடுக்கு LSTM செல்களை முயற்சிக்க வேண்டும் என்று தோன்றுகிறது. முந்தைய பிரிவில் நாம் குறிப்பிட்டது போல, LSTM-இன் ஒவ்வொரு அடுக்கும் உரையிலிருந்து குறிப்பிட்ட மாதிரிகளை எடுக்கிறது, மற்றும் எழுத்து-நிலை ஜெனரேட்டரின் சூழலில், கீழ் நிலை LSTM எழுத்துக்களைப் பொறுப்பாக எடுக்கும், மேலும் மேல் நிலைகள் - சொற்கள் மற்றும் சொற்களின் சேர்க்கைகளை. இதை LSTM கட்டமைப்புக்கு அடுக்குகளின் எண்ணிக்கை அளவுருவை அனுப்புவதன் மூலம் எளிதாக செயல்படுத்தலாம்.
* நீங்கள் **GRU யூனிட்கள்** மற்றும் **வெவ்வேறு மறைமுக அடுக்கு அளவுகள்** ஆகியவற்றுடன் பரிசோதிக்க விரும்பலாம். மிகப் பெரிய மறைமுக அடுக்கு அளவு overfitting-ஐ ஏற்படுத்தலாம் (எ.கா. நெட்வொர்க் சரியான உரையை கற்றுக்கொள்ளும்), மற்றும் சிறிய அளவு நல்ல முடிவை உருவாக்க முடியாது.


## மென்மையான உரை உருவாக்கம் மற்றும் வெப்பநிலை

முந்தைய `generate` வரையறையில், உருவாக்கப்பட்ட உரையில் அடுத்த எழுத்தாக எப்போதும் அதிக வாய்ப்புள்ள எழுத்தை எடுத்துக்கொண்டோம். இதனால், உரை அடிக்கடி ஒரே எழுத்து வரிசைகளில் மீண்டும் மீண்டும் "சுழல்கிறது", இதைப் போன்ற உதாரணத்தில்:
```
today of the second the company and a second the company ...
```

ஆனால், அடுத்த எழுத்துக்கான வாய்ப்பு விநியோகத்தைப் பார்க்கும்போது, சில அதிக வாய்ப்புகளுக்கிடையேயான வித்தியாசம் பெரியதாக இருக்காது, உதாரணமாக, ஒரு எழுத்துக்கு 0.2 வாய்ப்பு இருக்கலாம், மற்றொன்றுக்கு 0.19, போன்றவை. உதாரணமாக, '*play*' வரிசையில் அடுத்த எழுத்தைத் தேடும்போது, அடுத்த எழுத்து இடைவெளி அல்லது **e** (உதாரணமாக *player* என்ற வார்த்தையில்) இரண்டுமே சமமாக இருக்கலாம்.

இதனால், அதிக வாய்ப்புள்ள எழுத்தைத் தேர்ந்தெடுப்பது எப்போதும் "நியாயமானது" அல்ல என்பதை நாங்கள் உணர்கிறோம், ஏனெனில் இரண்டாவது அதிக வாய்ப்புள்ள எழுத்தைத் தேர்ந்தெடுத்தாலும் அர்த்தமுள்ள உரைக்கு வழிவகுக்கலாம். நெட்வொர்க் வெளியீட்டால் வழங்கப்படும் வாய்ப்பு விநியோகத்திலிருந்து எழுத்துகளை **மாதிரியாக** தேர்ந்தெடுப்பது புத்திசாலித்தனமாக இருக்கும்.

இந்த மாதிரியை **multinomial distribution** எனப்படும் பன்முக விநியோகத்தை செயல்படுத்தும் `multinomial` செயல்பாட்டைப் பயன்படுத்தி செய்யலாம். இந்த **மென்மையான** உரை உருவாக்கத்தை செயல்படுத்தும் ஒரு செயல்பாடு கீழே வரையறுக்கப்பட்டுள்ளது:


In [10]:
def generate_soft(net,size=100,start='today ',temperature=1.0):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            #nc = torch.argmax(out[0][-1])
            out_dist = out[0][-1].div(temperature).exp()
            nc = torch.multinomial(out_dist,1)[0]
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"--- Temperature = {i}\n{generate_soft(net,size=300,start='Today ',temperature=i)}\n")

--- Temperature = 0.3
Today and a company and complete an all the land the restrational the as a security and has provers the pay to and a report and the computer in the stand has filities and working the law the stations for a company and with the company and the final the first company and refight of the state and and workin

--- Temperature = 0.8
Today he oniis its first to Aus bomblaties the marmation a to manan  boogot that pirate assaid a relaid their that goverfin the the Cappets Ecrotional Assonia Cition targets it annight the w scyments Blamity #39;s TVeer Diercheg Reserals fran envyuil that of ster said access what succers of Dour-provelith

--- Temperature = 1.0
Today holy they a 11 will meda a toket subsuaties, engins for Chanos, they's has stainger past to opening orital his thempting new Nattona was al innerforder advan-than #36;s night year his religuled talitatian what the but with Wednesday to Justment will wemen of Mark CCC Camp as Timed Nae wome a leaders

--- Temper

நாங்கள் **வெப்பநிலை** எனப்படும் மேலும் ஒரு அளவுருவை அறிமுகப்படுத்தியுள்ளோம், இது மிக அதிகமான சாத்தியமானதை எவ்வளவு கடினமாக பின்பற்ற வேண்டும் என்பதை குறிக்க பயன்படுத்தப்படுகிறது. வெப்பநிலை 1.0 ஆக இருந்தால், நாங்கள் நியாயமான பன்முக மாதிரியைப் பயன்படுத்துகிறோம், மேலும் வெப்பநிலை முடிவில்லாத அளவுக்கு சென்றால் - அனைத்து சாத்தியங்களும் சமமாகிவிடும், மேலும் அடுத்த எழுத்தை சீரற்ற முறையில் தேர்ந்தெடுக்கிறோம். கீழே உள்ள உதாரணத்தில், வெப்பநிலையை அதிகமாக உயர்த்தும் போது உரை அர்த்தமற்றதாக மாறுவதை நாம் கவனிக்கலாம், மேலும் அது 0-க்கு அருகில் சென்றால் "சுழல்கின்ற" கடினமாக உருவாக்கப்பட்ட உரையை ஒத்ததாக இருக்கும்.



---

**குறிப்பு**:  
இந்த ஆவணம் [Co-op Translator](https://github.com/Azure/co-op-translator) என்ற AI மொழிபெயர்ப்பு சேவையைப் பயன்படுத்தி மொழிபெயர்க்கப்பட்டுள்ளது. நாங்கள் துல்லியத்திற்காக முயற்சிக்கின்றோம், ஆனால் தானியங்கி மொழிபெயர்ப்புகளில் பிழைகள் அல்லது தவறான தகவல்கள் இருக்கக்கூடும் என்பதை தயவுசெய்து கவனத்தில் கொள்ளவும். அதன் தாய்மொழியில் உள்ள மூல ஆவணம் அதிகாரப்பூர்வ ஆதாரமாக கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்பு பரிந்துரைக்கப்படுகிறது. இந்த மொழிபெயர்ப்பைப் பயன்படுத்துவதால் ஏற்படும் எந்த தவறான புரிதல்கள் அல்லது தவறான விளக்கங்களுக்கு நாங்கள் பொறுப்பல்ல.
